In [1]:
import pandas as pd
import numpy as np
import seaborn as sp
import matplotlib as mplot
from collections import defaultdict
from scipy import stats

In [2]:
weeksalesdb = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv' , dtype = {'Store': int})
features = pd.read_csv('features.csv')

In [3]:
features.drop(columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace = True)

In [ ]:
#converts weeksalesdb rows into dict and merging departments
agg_sales = defaultdict(int)
for ind, sale in weeksalesdb.iterrows():
    agg_sales[str(sale['Store'])+'.'+sale['Date']] += sale['Weekly_Sales']

In [ ]:
#converts back into db with updated sales value
storesales = pd.DataFrame(columns=['Store','Date','Weekly_Sales'])

for key, value in agg_sales.items():
    store, date = key.split('.')
    newrow = [store, date, value]
    storesales.loc[len(storesales)] = newrow

In [ ]:
#function to convert date to integer
def convert_date(date):
    date = date.strip()
    date = date.replace('-','')
    date = date.strip()
    return int(date)

#convert date columns
storesales['Date'] = storesales['Date'].apply(convert_date)
features['Date'] = features['Date'].apply(convert_date)

In [ ]:
#convert store column to int (was str)
storesales['Store'] = pd.to_numeric(storesales['Store'])
features.head(10)

In [ ]:
#exported csv files
storesales.to_csv('storesales.csv')
features.to_csv('features2.csv')

In [ ]:
#merged two dataframes together on the store and date column
merged = pd.merge(storesales, features, on =['Store', 'Date'], how = 'outer')

In [ ]:
#removed rows where unemployment data was empty
merged.dropna(subset = ['Unemployment'], inplace = True)

In [ ]:
#export cleaned features
merged.to_csv('features_clean.csv', index = False)

In [ ]:
merged.head(10)

In [ ]:
#checked for outliers in the weekly sales column, most of these are on or near holidays.
outliers = merged[merged['Weekly_Sales'] > merged['Weekly_Sales'].mean() + 3 * merged['Weekly_Sales'].std()]
outliers